In [1]:
#Instalaremos NIBABEL
#-q es para no mostrar el log de instalación
!pip install -q nilearn
!pip install -q nibabel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.7/12.7 MB 40.3 MB/s eta 0:00:00


In [3]:
from nilearn import image,plotting,datasets
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import interact, IntSlider ###Para mostrar un slider

##montar drive para que colab lo vea. Nota: pedirá permisos de usuario
from google.colab import drive
drive.mount('/content/drive')


###correr esta línea para chequear carpeta con la data
ls -R /content/drive/Mydrive

##Cambiar el directorio
import os
#os.chdir("/content/drive/MyDrive/Neurociencia_guatemala/visualization")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


NameError: name 'ls' is not defined

In [ ]:
#load data

#cargar la parcelación AAL: 116 áreas, 0-89 son cerebrales, 90-... es cerebelo
AAL_mask = nib.load("structural_data/AAL.nii")

##cargar un template estructural
mni_template = datasets.load_mni152_template()

##opcional: cargar dos estructuras subcorticales
lc_mask = nib.load("structural_data/LCmetaMask_bilateral_MNI05_s01f_plus50.nii")
bf_mask = nib.load("structural_data/BF-2005_MNI_resliced_to_1.5mm.nii")


In [ ]:
###########cargar nombres de áreas

##El orden original es LLLRRR. Lo cambiamos a LRLRLR (es lo más usual)
AAL_labels_unsorted = pd.read_csv("symmetrized_AAL_labels.txt")["label"].values ###labels as array
left = AAL_labels_unsorted[:45]
right = AAL_labels_unsorted[45:][::-1]


####the usual order is ...LRLRLR...
AAL_labels_sorted = np.zeros_like(AAL_labels_unsorted)
AAL_labels_sorted[::2] = left
AAL_labels_sorted[1::2] = right


##imprimir áreas en pantalla
# for i,area in enumerate(AAL_labels_sorted):
#   print(i,area)

In [ ]:
##ver todas las áreas del AAL

fig = plt.figure(1)
fig.clf()
display = plotting.plot_anat(
    bg_img=mni_template,
    title="MNI T1 template",
    colorbar=False,
    display_mode="xz",
    cut_coords = (50,0),   #######control cut
    black_bg=False,  # Set background to white
    figure=fig
)
plt.show()




In [ ]:
from ipywidgets import interact, IntSlider


##opcional: ver áreas del AAL o MNI
data = AAL_mask.get_fdata()  # or any NIfTI data
# data = mni_template.get_fdata()

colormap = 'gray'

##funcion que genera un heatmap por slice
def view_slice(z=0):
    plt.figure(figsize=(5,5))
    im=plt.imshow(np.rot90(data[:,:,z]), cmap=colormap)
    plt.axis('off')

    cbar = plt.colorbar(im, fraction=0.046, pad=0.04)
    cbar.set_label("Overlay value")

    plt.show()

interact(view_slice, z=IntSlider(min=0, max=data.shape[2]-1, step=1, value=data.shape[2]//2));

###ejercicio, cómo hacer esto de lado a lado?


In [ ]:
# Hacemos una copia para modificar
modified_data = AAL_mask.get_fdata().copy()
# Visualizar solo corteza
modified_data[np.isin(modified_data, np.unique(modified_data)[91:])] = 0 ######ONLY cortex
# Nueva imagen solo con corteza
AAL90_mask = nib.Nifti1Image(modified_data, affine=AAL_mask.affine, header=AAL_mask.header)



##Sobreponer areas del AAL
fig = plt.figure(2)
fig.clf()
display = plotting.plot_anat(
    bg_img=mni_template,
    title="AAL over MNI T1 template",
    colorbar=False,
    display_mode="xz",
    cut_coords = (0,0),   #######control cut
    black_bg=False,  # Set background to white
    figure=fig
)
display.add_overlay(AAL90_mask, cmap="Blues",vmin=0,vmax=1,transparency=0.6)
plt.show()

In [ ]:
####mostrar areas especificas

N_areas = np.array([76,77]) ##indices de areas

print(f"displaying areas: {AAL_labels_sorted[N_areas]}")


modified_data = AAL_mask.get_fdata().copy()
modified_data[~np.isin(modified_data, np.unique(modified_data)[N_areas+1])] = 0 ### +1 porque el primer elemento es el fondo (0)
to_display = nib.Nifti1Image(modified_data, affine=AAL_mask.affine, header=AAL_mask.header)



###mostrar
fig = plt.figure(2)
fig.clf()
display = plotting.plot_anat(
    bg_img=mni_template,
    title="AAL over MNI T1 template",
    colorbar=False,
    display_mode="xz",
    cut_coords = (0,1),   #######control cut
    black_bg=False,  # Set background to white
    figure=fig
)
display.add_overlay(to_display, cmap="Blues",vmin=0,vmax=1,transparency=0.6)
plt.show()

In [ ]:
#%% plotear areas subcorticales (LC y BF) junto con areas AAL

fig = plt.figure(5)
fig.clf()
display = plotting.plot_anat(
    bg_img=mni_template,
    title="BF and LC mask over T1 template",
    display_mode="xz",
    cut_coords = (4.7,-23),
    black_bg=False,  # Set background to white
    draw_cross = False,
    colorbar=False,
    figure=fig,
)
display.add_overlay(bf_mask, cmap="Greens",vmin=0,vmax=1.1)#, threshold=0.1, alpha=0.8)
display.add_overlay(lc_mask, cmap="Reds",vmin=0,vmax=1.15)#, threshold=0.1, alpha=0.8)
display.add_overlay(AAL90_mask, cmap="Blues",vmin=0,vmax=1,transparency=0.3)#, threshold=0.1, alpha=0.8)
# plt.savefig("LC_and_BF_over_mni.svg",dpi=300)
plt.show()

In [ ]:
##optional: load functional data!!
# data = image.mean_img("conc_data_brain_filt_reg_20090602AM.img", copy_header=True)
# fdata = data.get_fdata()
# print("Min:", np.min(fdata))
# print("Max:", np.max(fdata))
# print("Mean:", np.mean(fdata))

# fig = plt.figure(5)
# fig.clf()
# display= plotting.plot_epi(data, title="LC and BF mask over BOLD average",
#                   display_mode="xz",
#                   cut_coords = (4.7,-23),
#                    vmin=np.quantile(fdata[fdata!=0],0.01),
#                     vmax=np.quantile(fdata[fdata!=0],0.9),
#                     colorbar=False,
#                   figure=fig)

# display.add_overlay(bf_mask, cmap="Greens",vmin=0,vmax=1.3)#, threshold=0.1, alpha=0.8)
# display.add_overlay(lc_mask, cmap="Reds",vmin=0,vmax=1.15)#, threshold=0.1, alpha=0.8)
# display.add_overlay(AAL90_mask, cmap="Blues",vmin=0,vmax=1)#, threshold=0.1, alpha=0.8)

# # plt.savefig("masks_over_BOLD_20090602AM.svg",dpi=300)
# plt.show()

# halt




